In [83]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [53]:
df.drop(columns = ['id','Unnamed: 32'], inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [54]:
X = df.iloc[:,1:]
y = df.iloc[:,0]
X, y

(     radius_mean  texture_mean  ...  symmetry_worst  fractal_dimension_worst
 0          17.99         10.38  ...          0.4601                  0.11890
 1          20.57         17.77  ...          0.2750                  0.08902
 2          19.69         21.25  ...          0.3613                  0.08758
 3          11.42         20.38  ...          0.6638                  0.17300
 4          20.29         14.34  ...          0.2364                  0.07678
 ..           ...           ...  ...             ...                      ...
 564        21.56         22.39  ...          0.2060                  0.07115
 565        20.13         28.25  ...          0.2572                  0.06637
 566        16.60         28.08  ...          0.2218                  0.07820
 567        20.60         29.33  ...          0.4087                  0.12400
 568         7.76         24.54  ...          0.2871                  0.07039
 
 [569 rows x 30 columns],
 0      M
 1      M
 2      M
 3    

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.2)

In [56]:
X_train

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
482,13.470,14.06,87.32,546.3,0.10710,0.11550,0.05786,0.052660,0.1779,0.06639,0.1588,0.5733,1.102,12.84,0.004450,0.01452,0.01334,0.008791,0.01698,0.002787,14.830,18.32,94.94,660.2,0.1393,0.2499,0.18480,0.13350,0.3227,0.09326
247,12.890,14.11,84.95,512.2,0.08760,0.13460,0.13740,0.039800,0.1596,0.06409,0.2025,0.4402,2.393,16.35,0.005501,0.05592,0.08158,0.013700,0.01266,0.007555,14.390,17.70,105.00,639.1,0.1254,0.5849,0.77270,0.15610,0.2639,0.11780
61,8.598,20.98,54.66,221.8,0.12430,0.08963,0.03000,0.009259,0.1828,0.06757,0.3582,2.0670,2.493,18.39,0.011930,0.03162,0.03000,0.009259,0.03357,0.003048,9.565,27.04,62.06,273.9,0.1639,0.1698,0.09001,0.02778,0.2972,0.07712
378,13.660,15.15,88.27,580.6,0.08268,0.07548,0.04249,0.024710,0.1792,0.05897,0.1402,0.5417,1.101,11.35,0.005212,0.02984,0.02443,0.008356,0.01818,0.004868,14.540,19.64,97.96,657.0,0.1275,0.3104,0.25690,0.10540,0.3387,0.09638
487,19.440,18.82,128.10,1167.0,0.10890,0.14480,0.22560,0.119400,0.1823,0.06115,0.5659,1.4080,3.631,67.74,0.005288,0.02833,0.04256,0.011760,0.01717,0.003211,23.960,30.39,153.90,1740.0,0.1514,0.3725,0.59360,0.20600,0.3266,0.09009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,13.940,13.17,90.31,594.2,0.12480,0.09755,0.10100,0.066150,0.1976,0.06457,0.5461,2.6350,4.091,44.74,0.010040,0.03247,0.04763,0.028530,0.01715,0.005528,14.620,15.38,94.52,653.3,0.1394,0.1364,0.15590,0.10150,0.2160,0.07253
122,24.250,20.20,166.20,1761.0,0.14470,0.28670,0.42680,0.201200,0.2655,0.06877,1.5090,3.1200,9.807,233.00,0.023330,0.09806,0.12780,0.018220,0.04547,0.009875,26.020,23.99,180.90,2073.0,0.1696,0.4244,0.58030,0.22480,0.3222,0.08009
500,15.040,16.74,98.73,689.4,0.09883,0.13640,0.07721,0.061420,0.1668,0.06869,0.3720,0.8423,2.304,34.84,0.004123,0.01819,0.01996,0.010040,0.01055,0.003237,16.760,20.43,109.70,856.9,0.1135,0.2176,0.18560,0.10180,0.2177,0.08549
367,12.210,18.02,78.31,458.4,0.09231,0.07175,0.04392,0.020270,0.1695,0.05916,0.2527,0.7786,1.874,18.57,0.005833,0.01388,0.02000,0.007087,0.01938,0.001960,14.290,24.04,93.85,624.6,0.1368,0.2170,0.24130,0.08829,0.3218,0.07470


In [57]:
type(X_train)

pandas.DataFrame

In [58]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [60]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [61]:
X_train

array([[-0.20242127, -1.4162111 , -0.20692618, ...,  0.26345579,
         0.37645278,  0.59691985],
       [-0.39032141, -1.40330926, -0.31811133, ...,  0.63671636,
        -0.52097218,  2.10856877],
       [-1.78078248,  0.36940368, -1.73912318, ..., -1.48261095,
        -0.01273662, -0.39729423],
       ...,
       [ 0.30620499, -0.72467243,  0.32835758, ..., -0.26009998,
        -1.2260918 ,  0.11829263],
       [-0.61061813, -0.3943853 , -0.62961739, ..., -0.48323053,
         0.36271668, -0.54636474],
       [ 0.85694679,  0.52680614,  0.86176481, ...,  1.05291843,
         2.87336986,  0.17003611]], shape=(113, 30))

## Creating a class for the simple one neuron neural network 

In [ ]:
class mySimpleNet:
    def __init__(self,X):
        self.weights = torch.rand(X.shape[1],1, dtype = torch.float64, requires_grad= True)
        self.bias = torch.rand(1,dtype=torch.float64,requires_grad=True)

    #Forward pass
    def forward_pass(self, X):
        z= torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
        
    #Loss Function
    def loss(self,y_pred,y):
        epsilon = 1e-6
        y_pred = torch.clamp(y_pred,epsilon,1-epsilon)
        loss = -(y*torch.log(y_pred)+(1-y)*torch.log(1-y_pred)).mean()
        return loss

### Training Parameters

In [76]:
learning_rate = 0.1
epoch = 2500

In [77]:
model = mySimpleNet(X_train_tensor)

In [80]:
for epoch in range(epoch):
    #Forward Pass
    y_pred = model.forward_pass(X_train_tensor)
    #Calculate Loss
    loss = model.loss(y_pred, y_train_tensor)
    #Backward Pass
    loss.backward()
    #Update Weights and Biases
    with torch.no_grad():
        model.weights -= learning_rate*model.weights.grad
        model.bias -= learning_rate*model.bias.grad
    #Reset the Gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    print(f'Epoch: {epoch+1} is {loss.item()}')

Epoch: 1 is 0.6727672008876263
Epoch: 2 is 0.6750629382670699
Epoch: 3 is 0.674947847926037
Epoch: 4 is 0.6748380207639173
Epoch: 5 is 0.6747332200965774
Epoch: 6 is 0.6746332196769357
Epoch: 7 is 0.6745378031921992
Epoch: 8 is 0.674446763819896
Epoch: 9 is 0.6743599038186957
Epoch: 10 is 0.6742770341428651
Epoch: 11 is 0.6741979740750635
Epoch: 12 is 0.6741225508748423
Epoch: 13 is 0.6740505994414258
Epoch: 14 is 0.6739819619899023
Epoch: 15 is 0.6739164877402022
Epoch: 16 is 0.6738540326183651
Epoch: 17 is 0.6737944589696541
Epoch: 18 is 0.6737376352830947
Epoch: 19 is 0.6736834359270522
Epoch: 20 is 0.6736317408954562
Epoch: 21 is 0.6735824355642995
Epoch: 22 is 0.6735354104580511
Epoch: 23 is 0.6734905610256221
Epoch: 24 is 0.6734477874255436
Epoch: 25 is 0.6734069943200138
Epoch: 26 is 0.6733680906774918
Epoch: 27 is 0.6733309895835139
Epoch: 28 is 0.6732956080594259
Epoch: 29 is 0.6732618668887286
Epoch: 30 is 0.6732296904507454
Epoch: 31 is 0.6731990065613291
Epoch: 32 is 0.6731

## Evaluation

In [ ]:
with torch.no_grad():
    y_pred = model.forward_pass(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test).float().mean()
    print(f'Accuracy: {accuracy}')